В дополнительных заданиях можно реализовывать логику только частично, тогда
баллы будут снижены. На оценку за каждое задание также влияет корректность шагов и
качество кода.
Задания под * не являются обязательными, но будут учитываться при отборе
участников.

Рекомендации к оборудованию и ПО
• Для эффективной работы тебе потребуется ПК / ноутбук.
• Если с работой программы возникнут проблемы, пиши на shift_support@cft.ru
• Установи и подготовь к работе ПО: Docker, Jupyter Notebook, IDE PyCharm, IntelliJIDEA

Задание
Необходимо реализовать ETL1 пайплайн (В данном случае приложение на Scala, Java
или Python), которое загружает исходные данные, трансформирует их, а затем выгружает)
для выгрузки данных прогнозов погоды.
Для получения данных будет использован https://open-meteo.com/

Пример запроса данных по API open-meteo:
https://api.openmeteo.com/v1/forecast?latitude=55.0344&longitude=82.9434&daily=sunrise,sunset,daylight_duration&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,temperature_80m,temperature_120m,wind_speed_10m,wind_speed_80m,wind_direction_10m,wind_direction_80m,visibility,evapotranspiration,weather_code,soil_temperature_0cm,soil_temperature_6cm,rain,showers,snowfall&timezone=auto&timeformat=unixtime&wind_speed_unit=kn&temperature_unit=fahrenheit&precipitation_unit=inch&start_date=2025-05-16&end_date=2025-05-30

Описание данных JSON в приложении, таблица "Описание данных JSON"

Задание 1: Наивная реализация
Реализовать приложение, которое будет делать следующее:
1. считывать json в формате API open-meteo, json взять из запроса в
"Пример запроса данных по API open-meteo"
2. Выполнять трансформации данных, которые на Ваш взгляд необходимы для
получения данных в формате для выгрузки - т.е в формате, как в таблице
"Итоговая таблица" из приложения.
3. Сохранить данные в csv. Поля, необходимые в csv и их описание представлены в
таблице "Итоговая таблица" из приложения.

In [ ]:
import requests
import json

base_url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 55.0344,
    "longitude": 82.9434,
    "daily": "sunrise,sunset,daylight_duration",
    "hourly": (        
        "temperature_2m,relative_humidity_2m,dew_point_2m,"
        "apparent_temperature,temperature_80m,temperature_120m,"
        "wind_speed_10m,wind_speed_80m,wind_direction_10m,"
        "wind_direction_80m,visibility,evapotranspiration,"
        "weather_code,soil_temperature_0cm,soil_temperature_6cm,"
        "rain,showers,snowfall"
        ),
    "timezone": "auto",
    "start_date": "2025-05-16",
    "end_date": "2025-05-30"
}

response = requests.get(base_url, params=params)
meteo_data = response.json()
print(meteo_data)  

with open('weather_data.json', 'w', encoding='utf-8') as f:
    json.dump(meteo_data, f, ensure_ascii=False, indent=4)



print("Status Code:", response.status_code)  
print("Response Text:", response.text[:200])  


{'latitude': 55.0, 'longitude': 83.0, 'generationtime_ms': 0.1811981201171875, 'utc_offset_seconds': 25200, 'timezone': 'Asia/Novosibirsk', 'timezone_abbreviation': 'GMT+7', 'elevation': 135.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'dew_point_2m': '°C', 'apparent_temperature': '°C', 'temperature_80m': '°C', 'temperature_120m': '°C', 'wind_speed_10m': 'km/h', 'wind_speed_80m': 'km/h', 'wind_direction_10m': '°', 'wind_direction_80m': '°', 'visibility': 'm', 'evapotranspiration': 'mm', 'weather_code': 'wmo code', 'soil_temperature_0cm': '°C', 'soil_temperature_6cm': '°C', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'hourly': {'time': ['2025-05-30T00:00', '2025-05-30T01:00', '2025-05-30T02:00', '2025-05-30T03:00', '2025-05-30T04:00', '2025-05-30T05:00', '2025-05-30T06:00', '2025-05-30T07:00', '2025-05-30T08:00', '2025-05-30T09:00', '2025-05-30T10:00', '2025-05-30T11:00', '2025-05-30T12:00', '2025-05-30T13:00', '2025-05-30T14:00', '20

Вычисляем среднюю температуру  avg_temperature_2m_24h

In [3]:
import json
import numpy as np # используется для более эффективного разбиения и вычисления avg

with open('weather_data_main.json', 'r', encoding='utf-8') as f:
    meteo_data = json.load(f)


def calculating_avg_24(data, chunk_size):
    '''
    Функция для разбеения списка на подсписки длины chunk 
    и вычисления среднего значения
    '''
    
    list_avg_values = []

    arr = np.array(data)
    chunks = np.array_split(arr, len(data) // chunk_size)

    for chunk in chunks:
        if len(chunk) >= (chunk_size - 1):
            avg = chunk.mean()
            list_avg_values.append(avg)
            #print(list_avg_values)
        else:
            print(f'len_chunk = {len(chunk)}')
    return list_avg_values
        

print(calculating_avg_24(meteo_data['hourly']['temperature_2m'], 24))

[np.float64(9.733333333333334), np.float64(11.633333333333335), np.float64(12.958333333333334), np.float64(13.558333333333335), np.float64(16.745833333333334), np.float64(19.537499999999998), np.float64(21.0625), np.float64(22.4375), np.float64(16.775000000000002), np.float64(12.241666666666667), np.float64(9.054166666666667), np.float64(4.266666666666667), np.float64(5.033333333333333), np.float64(7.720833333333334), np.float64(13.541666666666666)]


In [ ]:
final_list = (
    "temperature_2m", "relative_humidity_2m", "dew_point_2m",
    "apparent_temperature", "temperature_80m", "temperature_120m",
    "wind_speed_10m", "wind_speed_80m","visibility", 
    "rain", "showers", "snowfall"    
)

#print(final_list)

for parametr in final_list:
    final_param = calculating_avg_24(meteo_data['hourly'][parametr], 24)
    print(final_param)

print(type(final_list))

('temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'temperature_80m', 'temperature_120m', 'wind_speed_10m', 'wind_speed_80m', 'visibility', 'rain', 'showers', 'snowfall')
<class 'tuple'>
